In [1]:
%conda install pandas
%conda install statsmodels
%conda install scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update

In [152]:
import pandas as pd
industries_data = pd.read_csv('/Users/jingzhao/Desktop/FE PW1/Data/17 Industry Portfolios.CSV')
financial_data = pd.read_csv('/Users/jingzhao/Desktop/FE PW1/Data/Financial Data.CSV')

In [153]:
# HML: Lasso coefficient
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

merged_data = pd.merge(industries_data, financial_data[['Date', 'HML']], on='Date')
X = merged_data.drop(columns=['Date', 'HML'])
y = merged_data['HML']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Using LASSO regression with cross-validation to find the best alpha
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Extracting the coefficients and the selected features
lasso_coefs = lasso.coef_
selected_features = X.columns[np.abs(lasso_coefs) > 0]

selected_features, mse, lasso.alpha_

# Creating a DataFrame to display the coefficients and their corresponding features
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_coefs
})

# Filtering out the features with zero coefficients
significant_coefficients = coefficients[np.abs(coefficients['Coefficient']) > 0]
significant_coefficients.sort_values(by='Coefficient', ascending=False)

print(selected_features)

Index(['Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems', 'Cnsum', 'Cnstr',
       'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils', 'Rtail', 'Finan',
       'Other'],
      dtype='object')


In [151]:
# HML Rolling Regression
from datetime import timedelta
import pandas as pd
from sklearn.linear_model import LinearRegression

def rolling_regression(data, selected_features, train_years, test_years):
    # Convert 'Date' to datetime for easier date manipulation
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

    # Get the unique years in the dataset
    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        # Define training and testing periods
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + pd.DateOffset(years=train_years)
        test_end = train_end + pd.DateOffset(years=test_years)

        # Subset the data for training and testing
        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['HML']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_HML': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, selected_features, train_years=5, test_years=5)
results_10_year = rolling_regression(merged_data, selected_features, train_years=10, test_years=5)
results_20_year = rolling_regression(merged_data, selected_features, train_years=20, test_years=5)

print(model.coef)

KeyError: 'HML'

In [130]:
# Beta and HML
def extract_betas_and_predictions(results):
    betas = []
    predicted_HML = []

    for index, row in results.iterrows():
        # Extract beta coefficients
        betas.append({
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end'],
            **{f'beta_{i}': coef for i, coef in enumerate(row['coefficients'])}
        })
        
        # Extract predicted HML values
        for prediction in row['predicted_HML']:
            predicted_HML.append({
                'test_end': row['test_end'],
                'predicted_HML': prediction
            })

    betas_df = pd.DataFrame(betas)
    predicted_HML_df = pd.DataFrame(predicted_HML)

    return betas_df, predicted_HML_df

# Process the results for each time scheme
betas_5_year, predicted_HML_5_year = extract_betas_and_predictions(results_5_year)
betas_10_year, predicted_HML_10_year = extract_betas_and_predictions(results_10_year)
betas_20_year, predicted_HML_20_year = extract_betas_and_predictions(results_20_year)

df = pd.DataFrame(betas_5_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_Beta_5yr.csv', index=False)
df = pd.DataFrame(betas_10_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_Beta_10yr.csv', index=False)
df = pd.DataFrame(betas_20_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_Beta_20yr.csv', index=False)
df = pd.DataFrame(predicted_HML_5_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_5yr.csv', index=False)
df = pd.DataFrame(predicted_HML_10_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_10yr.csv', index=False)
df = pd.DataFrame(predicted_HML_20_year)
df.to_csv('/Users/jingzhao/Desktop/Lasso_HML_20yr.csv', index=False)

In [134]:
# Residual
def calculate_residuals(data, predicted_HML, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate residuals
        predicted_HML_values = predicted_HML['predicted_HML'].iloc[predicted_index:predicted_index + len(actual_HML)]
        residual = actual_HML - predicted_HML_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_HML)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_HML_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_HML_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_HML_20_year, 20, 5)
print(residuals_5_year)

      residuals
0      5.150271
1     -2.417107
2      1.244794
3      3.351280
4      5.916826
...         ...
1075   1.277399
1076  -1.560172
1077  -2.308562
1078   3.377945
1079   1.691241

[1080 rows x 1 columns]


In [135]:
# Residual Summary
residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean     -0.144324
 std       2.627564
 min     -13.519619
 25%      -1.541285
 50%      -0.288517
 75%       1.280076
 max      18.252588,
          residuals
 count  1020.000000
 mean     -0.122225
 std       2.129064
 min     -10.531912
 25%      -1.355265
 50%      -0.202789
 75%       1.045957
 max       9.826449,
         residuals
 count  900.000000
 mean    -0.173789
 std      2.025429
 min     -8.039950
 25%     -1.424272
 50%     -0.282700
 75%      0.934058
 max      6.785557)

In [136]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(7472.013824415835, 4634.276112332735, 3715.206031934155)

In [140]:
def calculate_out_of_sample_r_squared(data, predicted_HML, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    for start_year in unique_years:
        if start_year + train_years + test_years > unique_years[-1]:
            break

        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_HML = test_data['HML'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_HML = np.mean(actual_HML)
        total_sum_squares += np.sum((actual_HML - mean_actual_HML) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_HML)] ** 2)

        # Update indices
        predicted_index += len(actual_HML)

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_HML_20_year, residuals_20_year, 20, 5)
r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.84759175456176, 0.8779221245139004, 0.8818784218952873)

In [141]:
# MOM Select Regressor
merged_data = pd.merge(industries_data, financial_data[['Date', 'MOM']], on='Date')
X = merged_data.drop(columns=['Date', 'MOM'])
y = merged_data['MOM']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Using LASSO regression with cross-validation to find the best alpha
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Extracting the coefficients and the selected features
lasso_coefs = lasso.coef_
selected_features = X.columns[np.abs(lasso_coefs) > 0]

selected_features, mse, lasso.alpha_

# Creating a DataFrame to display the coefficients and their corresponding features
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_coefs
})

# Filtering out the features with zero coefficients
significant_coefficients = coefficients[np.abs(coefficients['Coefficient']) > 0]
significant_coefficients.sort_values(by='Coefficient', ascending=False)

print(selected_features)

Index(['Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Cnsum', 'Cnstr', 'Steel',
       'FabPr', 'Machn', 'Cars ', 'Trans', 'Rtail', 'Finan', 'Other'],
      dtype='object')


In [144]:
# MOM Rolling Regression
def rolling_regression(data, selected_features, train_years, test_years):
    # Convert 'Date' to datetime for easier date manipulation
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')

    # Get the unique years in the dataset
    unique_years = data['Date'].dt.year.unique()
    
    results = []
    start_year = unique_years[0]

    # Perform rolling regression
    while start_year + train_years + test_years <= unique_years[-1]:
        # Define training and testing periods
        train_start = pd.Timestamp(year=start_year, month=1, day=1)
        train_end = train_start + pd.DateOffset(years=train_years)
        test_end = train_end + pd.DateOffset(years=test_years)

        # Subset the data for training and testing
        train_data = data[(data['Date'] >= train_start) & (data['Date'] < train_end)]
        test_data = data[(data['Date'] >= train_end) & (data['Date'] < test_end)]

        # Fit the model
        X_train = train_data[selected_features]
        y_train = train_data['MOM']
        model = LinearRegression().fit(X_train, y_train)

        # Predict on test data
        X_test = test_data[selected_features]
        y_pred = model.predict(X_test)

        # Collect coefficients and predictions
        coefficients = model.coef_
        results.append({
            'train_start': train_start,
            'train_end': train_end,
            'test_end': test_end,
            'coefficients': coefficients,
            'predicted_MOM': y_pred
        })

        # Move to the next period
        start_year += test_years

    return pd.DataFrame(results)

# Rolling regression for each scenario
results_5_year = rolling_regression(merged_data, selected_features, train_years=5, test_years=5)
results_10_year = rolling_regression(merged_data, selected_features, train_years=10, test_years=5)
results_20_year = rolling_regression(merged_data, selected_features, train_years=20, test_years=5)

In [145]:
# Beta and MOM
def extract_betas_and_predictions(results):
    betas = []
    predicted_MOM = []

    for index, row in results.iterrows():
        # Extract beta coefficients
        betas.append({
            'train_start': row['train_start'],
            'train_end': row['train_end'],
            'test_end': row['test_end'],
            **{f'beta_{i}': coef for i, coef in enumerate(row['coefficients'])}
        })
        
        # Extract predicted MOM values
        for prediction in row['predicted_MOM']:
            predicted_MOM.append({
                'test_end': row['test_end'],
                'predicted_MOM': prediction
            })

    betas_df = pd.DataFrame(betas)
    predicted_MOM_df = pd.DataFrame(predicted_MOM)

    return betas_df, predicted_MOM_df

# Process the results for each time scheme
betas_5_year, predicted_MOM_5_year = extract_betas_and_predictions(results_5_year)
betas_10_year, predicted_MOM_10_year = extract_betas_and_predictions(results_10_year)
betas_20_year, predicted_MOM_20_year = extract_betas_and_predictions(results_20_year)

In [146]:
# Residual
def calculate_residuals(data, predicted_MOM, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()
    residuals = []

    start_year = unique_years[0]
    predicted_index = 0

    while start_year + train_years + test_years <= unique_years[-1]:
        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate residuals
        predicted_MOM_values = predicted_MOM['predicted_MOM'].iloc[predicted_index:predicted_index + len(actual_MOM)]
        residual = actual_MOM - predicted_MOM_values
        residuals.extend(residual)

        # Update indices
        predicted_index += len(actual_MOM)
        start_year += test_years

    return pd.DataFrame({'residuals': residuals})

# Calculate residuals for each rolling scheme
residuals_5_year = calculate_residuals(merged_data, predicted_MOM_5_year, 5, 5)
residuals_10_year = calculate_residuals(merged_data, predicted_MOM_10_year, 10, 5)
residuals_20_year = calculate_residuals(merged_data, predicted_MOM_20_year, 20, 5)

In [147]:
# Residual summaries
residual_summary_5_year = residuals_5_year.describe()
residual_summary_10_year = residuals_10_year.describe()
residual_summary_20_year = residuals_20_year.describe()

residual_summary_5_year, residual_summary_10_year, residual_summary_20_year

(         residuals
 count  1080.000000
 mean      0.488007
 std       4.920040
 min     -35.485383
 25%      -1.745210
 50%       0.716320
 75%       3.334757
 max      16.416958,
          residuals
 count  1020.000000
 mean      0.403584
 std       4.389237
 min     -25.236305
 25%      -1.549850
 50%       0.569789
 75%       2.989516
 max      14.434734,
         residuals
 count  900.000000
 mean     0.240018
 std      4.140386
 min    -26.982898
 25%     -1.628540
 50%      0.294478
 75%      2.526104
 max     16.632041)

In [148]:
# SSE for each time scheme
sse_5_year = np.sum(residuals_5_year['residuals'] ** 2)
sse_10_year = np.sum(residuals_10_year['residuals'] ** 2)
sse_20_year = np.sum(residuals_20_year['residuals'] ** 2)

sse_5_year, sse_10_year, sse_20_year

(26376.327655018493, 19797.583460893133, 15463.220625138372)

In [150]:

def calculate_out_of_sample_r_squared(data, predicted_MOM, residuals, train_years, test_years):
    data['Date'] = pd.to_datetime(data['Date'], format='%Y%m')
    unique_years = data['Date'].dt.year.unique()

    total_sum_squares = 0
    residual_sum_squares = 0
    predicted_index = 0

    for start_year in unique_years:
        if start_year + train_years + test_years > unique_years[-1]:
            break

        # Define the testing period
        test_start = pd.Timestamp(year=start_year + train_years, month=1, day=1)
        test_end = test_start + timedelta(days=365 * test_years)

        # Subset the actual data for the testing period
        test_data = data[(data['Date'] >= test_start) & (data['Date'] < test_end)]
        actual_MOM = test_data['MOM'].values

        # Calculate total sum of squares and residual sum of squares
        mean_actual_MOM = np.mean(actual_MOM)
        total_sum_squares += np.sum((actual_MOM - mean_actual_MOM) ** 2)
        residual_sum_squares += np.sum(residuals['residuals'].iloc[predicted_index:predicted_index + len(actual_MOM)] ** 2)

        # Update indices
        predicted_index += len(actual_MOM)

    return 1 - (residual_sum_squares / total_sum_squares)

# Calculate out-of-sample R-squared for each rolling scheme
r_squared_5_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_5_year, residuals_5_year, 5, 5)
r_squared_10_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_10_year, residuals_10_year, 10, 5)
r_squared_20_year = calculate_out_of_sample_r_squared(merged_data, predicted_MOM_20_year, residuals_20_year, 20, 5)
r_squared_5_year, r_squared_10_year, r_squared_20_year

(0.7051126639018661, 0.7332057629178734, 0.7606087359070746)